### Imports

In [1]:
from dataclasses import dataclass
from typing import Dict, Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import numpy as np

from sklearn.metrics import roc_auc_score
from scipy.special import softmax

from medmnist import INFO

import tqdm

# import matplotlib.pyplot as plt
# import pandas as pd

import warnings
warnings.filterwarnings("ignore")

### Check details of 2 channel medmnist images

I would suggest this 2 datasets only for baseline experimenation. 

#### 2 Channels

In [2]:
dataset_path = "/home/localssk23/.medmnist/pneumoniamnist.npz"
data = np.load(dataset_path)
image = data['train_images'][0]
print(image.shape)
print(np.unique(image))

(28, 28)
[  0   2   3   4   5   6   8  11  19  21  23  25  27  28  37  42  45  46
  49  53  55  57  60  62  64  70  77  79  81  83  84  85  86  87  89  90
  91  92  93  94  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200
 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218
 219 220 221 224 225]


#### 3 Channels

In [3]:
dataset_path = "/home/localssk23/.medmnist/retinamnist.npz"
data = np.load(dataset_path)
image = data['train_images'][0]
print(image.shape)
print(np.unique(image))

(28, 28, 3)
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168]


# Dataset Definition

In [4]:
class MNISTPerturbedDataset(Dataset):
    def __init__(self, images, labels, perturbation_percentage, transform=None):
        """
        Args:
            images: Original image dataset
            labels: Corresponding labels
            perturbation_percentage: Percentage of pixels to perturb (0-100)
            transform: Optional transform to be applied on the perturbed image
        """
        self.images = images
        self.labels = labels
        self.perturbation_percentage = perturbation_percentage
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        # Apply percentage-wise perturbation
        perturbed_image = self.perturb_image_percentage(image, self.perturbation_percentage)
        
        if self.transform:
            perturbed_image = self.transform(perturbed_image)
            
        return perturbed_image, label
    
    def perturb_image_percentage(self, img, percent):
        # Calculate number of pixels to perturb
        total_pixels = img.shape[0] * img.shape[1]
        
        if percent is None or percent == 0:
            num_pixels = 0
        elif percent == 'one': #! Single pixel perturbation
            num_pixels = 1
        else:
            num_pixels = int(total_pixels * percent / 100)
        
        # Create a copy of the image
        perturbed_img = img.copy()
        
        # Generate random pixel positions
        positions = np.random.choice(total_pixels, num_pixels, replace=False)
        x_positions = positions // img.shape[1]
        y_positions = positions % img.shape[1]
        
        # Generate random values for selected pixels
        if img.ndim == 3:
            values = np.random.randint(0, 255, size=(num_pixels, img.shape[2]))
        else:
            values = np.random.randint(0, 255, size=num_pixels)
        
        # Modify the pixels
        for i in range(num_pixels):
            if img.ndim == 3:
                perturbed_img[x_positions[i], y_positions[i]] = values[i]
            else:
                perturbed_img[x_positions[i], y_positions[i]] = values[i]
                
        return perturbed_img

################################################################
############# 1 pixel perturbation orig paper code #############
################################################################
#? Paper: arxiv.org/abs/1710.08864
#? Code: https://github.com/Hyperparticle/one-pixel-attack-keras/blob/master/1_one-pixel-attack-cifar10.ipynb

# def perturb_image(xs, img):
#     # If this function is passed just one perturbation vector,
#     # pack it in a list to keep the computation the same
#     if xs.ndim < 2:
#         xs = np.array([xs])
    
#     # Copy the image n == len(xs) times so that we can 
#     # create n new perturbed images
#     tile = [len(xs)] + [1]*(xs.ndim+1)
#     imgs = np.tile(img, tile)
    
#     # Make sure to floor the members of xs as int types
#     xs = xs.astype(int)
    
#     height, width = img.shape[:2]
    
#     for x, img in zip(xs, imgs):
#         # Split x into an array of 5-tuples (perturbation pixels)
#         # i.e., [[x,y,r,g,b], ...] for RGB or [[x,y,value], ...] for BW
#         pixels = np.split(x, len(x) // (2 + CONFIG['num_channels']))
#         for pixel in pixels:
#             x_pos, y_pos, *values = pixel
#             # Add bounds checking
#             x_pos = np.clip(x_pos, 0, height - 1)
#             y_pos = np.clip(y_pos, 0, width - 1)
            
#             if CONFIG['num_channels'] == 1:
#                 img[x_pos, y_pos] = values[0]
#             else:
#                 img[x_pos, y_pos] = values
    
#     return imgs

class MNISTGaussianDataset(Dataset):
    def __init__(self, images, labels, noise_std, transform=None):
        """
        Args:
            images: Original image dataset
            labels: Corresponding labels
            noise_std: Standard deviation of the Gaussian noise to be added
            transform: Optional transform to be applied on the noisy image
        """
        self.images = images
        self.labels = labels
        self.noise_std = noise_std
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        # Apply Gaussian noise
        noisy_image = self.add_gaussian_noise(image, self.noise_std)
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            
        return noisy_image, label

    def add_gaussian_noise(self, image, noise_percent):
        # Convert percentage to standard deviation based on image range
        noise_std = (noise_percent/100.0) * np.max(image)
        
        # Generate noise
        noise = np.random.normal(0, noise_std, image.shape)
        
        # Add noise and clip to valid range
        noisy_image = np.clip(image + noise, 0, 255)
        
        return noisy_image.astype(np.uint8)
    

class MNISTRemoveDataset(Dataset):
    def __init__(self, images, labels, perturbation_percentage, transform=None):
        """
        Args:
            images: Original image dataset
            labels: Corresponding labels
            perturbation_percentage: Percentage of pixels to remove (0-100)
            transform: Optional transform to be applied on the perturbed image
        """
        self.images = images
        self.labels = labels
        self.perturbation_percentage = perturbation_percentage
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        # Apply percentage-wise perturbation
        perturbed_image = self.remove_pixel_percentage(image, self.perturbation_percentage)
        
        if self.transform:
            perturbed_image = self.transform(perturbed_image)
            
        return perturbed_image, label
    
    def remove_pixel_percentage(self, img, percent):
        # Calculate number of pixels to perturb
        total_pixels = img.shape[0] * img.shape[1]
        
        if percent is None or percent == 0:
            num_pixels = 0
        elif percent == 'one': #! Single pixel perturbation
            num_pixels = 1
        else:
            num_pixels = int(total_pixels * percent / 100)
        
        # Create a copy of the image
        perturbed_img = img.copy()
        
        # Generate random pixel positions
        positions = np.random.choice(total_pixels, num_pixels, replace=False)
        x_positions = positions // img.shape[1]
        y_positions = positions % img.shape[1]
        
        # Set selected pixels to 0
        if img.ndim == 3:
            values = np.zeros((num_pixels, img.shape[2]), dtype=img.dtype)
        else:
            values = np.zeros(num_pixels, dtype=img.dtype)
        
        # Modify the pixels
        for i in range(num_pixels):
            if img.ndim == 3:
                perturbed_img[x_positions[i], y_positions[i]] = values[i]
            else:
                perturbed_img[x_positions[i], y_positions[i]] = values[i]
                
        return perturbed_img

# Model

In [5]:
class Net_28(nn.Module): #? Notice GroupNorm Used.
    def __init__(self, in_channels, num_classes):
        super(Net_28, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3),
            nn.GroupNorm(4, 16),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 16, kernel_size=3),
            nn.GroupNorm(4, 16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3),
            nn.GroupNorm(16, 64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.GroupNorm(16, 64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.GroupNorm(16, 64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# class Net_224(nn.Module):
#     def __init__(self, in_channels, num_classes):
#         super(Net_224, self).__init__()

#         self.layer1 = nn.Sequential(
#             nn.Conv2d(in_channels, 16, kernel_size=3),
#             nn.GroupNorm(4, 16),
#             nn.ReLU())

#         self.layer2 = nn.Sequential(
#             nn.Conv2d(16, 16, kernel_size=3),
#             nn.GroupNorm(4, 16),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))

#         self.layer3 = nn.Sequential(
#             nn.Conv2d(16, 64, kernel_size=3),
#             nn.GroupNorm(16, 64),
#             nn.ReLU())

#         self.layer4 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3),
#             nn.GroupNorm(16, 64),
#             nn.ReLU())

#         self.layer5 = nn.Sequential(
#             nn.Conv2d(64, 64, kernel_size=3, padding=1),
#             nn.GroupNorm(16, 64),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))

#         self.layer6 = nn.Sequential(
#             nn.Conv2d(64, 128, kernel_size=3),
#             nn.GroupNorm(32, 128),
#             nn.ReLU())

#         self.layer7 = nn.Sequential(
#             nn.Conv2d(128, 128, kernel_size=3, padding=1),
#             nn.GroupNorm(32, 128),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))

#         self.fc = nn.Sequential(
#             nn.Linear(128 * 25 * 25, 256),
#             nn.ReLU(),
#             nn.Linear(256, 256),
#             nn.ReLU(),
#             nn.Linear(256, num_classes))

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
#         x = self.layer5(x)
#         x = self.layer6(x)
#         x = self.layer7(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x

# Metrics

In [6]:
def compute_auc(all_targets, all_outputs, task):
    all_targets, all_outputs = np.array(all_targets), np.array(all_outputs)
    if task == 'multi-label, binary-class':
        return roc_auc_score(all_targets, all_outputs, average='macro')
    elif all_outputs.shape[1] == 2:
        return roc_auc_score(all_targets, all_outputs[:, 1])
    else:
        softmax_outputs = softmax(all_outputs, axis=1)
        return roc_auc_score(all_targets, softmax_outputs, multi_class='ovr', average='macro')

### Configs

In [7]:
HOME = '/home/localssk23/'

dataset = 'retinamnist'

CONFIG = {
   "batch_size": 2,
   "num_epochs": 1,

   "device": torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
   
   "data_path": f'{HOME}.medmnist/{dataset}.npz',
   "result_path": f'{HOME}localdp/classification/results/',

   "num_folds": 1
}

DATASET = {
    'Perturbed': MNISTPerturbedDataset,
    'Removal': MNISTRemoveDataset,
    'Gaussian': MNISTGaussianDataset
}

PERTURBATION_VALUES = {
        'original': None,
        'noisy_1pixel': 'one',
        'noisy_1%': 1,
        'noisy_5%': 5,
        'noisy_10%': 10,
        'noisy_25%': 25,
        'noisy_50%': 50,
        'noisy_75%': 75,
        'noisy_90%': 90,
        'noisy_99%': 99
    }

REMOVAL_VALUES = {
        'original': None,
        'remove_1pixel': 'one',
        'remove_1%': 1,
        'remove_5%': 5,
        'remove_10%': 10,
        'remove_25%': 25,
        'remove_50%': 50,
        'remove_75%': 75,
        'remove_90%': 90,
        'remove_99%': 99
    }

GAUSSIAN_STD_VALUES = {
    'gaussian_1': 1,
    'gaussian_5': 5,
    'gaussian_10': 10,
    'gaussian_25': 25,
    'gaussian_50': 50,
    'gaussian_75': 75,
    'gaussian_90': 90,
    'gaussian_99': 99
    }

data_flag = CONFIG['data_path'].split('/')[-1].split('.')[0]
info = INFO[data_flag]

device = CONFIG['device']

CONFIG['num_classes'] = len(info['label'])
CONFIG['num_channels'] = info['n_channels']
CONFIG['task'] = info['task']

# Runners

#### Train

In [8]:
def lr_lambda(epoch):
    initial_lr = 0.001  # Initial learning rate
    if epoch < 50:
        return initial_lr / initial_lr  # Learning rate remains 0.001
    elif epoch < 75:
        return 0.1 * initial_lr / initial_lr  # Delay learning rate to 0.0001 after 50 epochs
    else:
        return 0.01 * initial_lr / initial_lr  # Delay learning rate to 0.00001 after 75 epochs

def train(model, train_loader, task):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    all_targets = []
    all_outputs = []

    if task == "multi-label, binary-class":
        criterion = nn.BCEWithLogitsLoss()
    else:
        criterion = nn.CrossEntropyLoss() #! Change to MSE (or just add it)
    criterion.to(device)

    lr = 0.001
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = LambdaLR(optimizer, lr_lambda)

    for epoch in tqdm.tqdm(range(CONFIG['num_epochs'])):
        for inputs, targets in train_loader:
            inputs, targets = inputs.float().to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                loss = criterion(outputs, targets)
                
                # Calculate accuracy for multi-label
                predicted = (outputs > 0.5).float()
                correct += (predicted == targets).all(dim=1).sum().item()
            else:
                targets = targets.squeeze().long()
                loss = criterion(outputs, targets)
                
                # Calculate accuracy for standard classification
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()

            total += targets.size(0)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

            # Store targets and outputs for AUC calculation
            all_targets.extend(targets.cpu().numpy())
            all_outputs.extend(outputs.detach().cpu().numpy())

    scheduler.step()

    return model

#### Test

In [9]:
def test(model, test_loader, task):
    model.eval()
    correct = 0
    total = 0
    all_targets = []
    all_outputs = []
    class_correct = {}
    class_total = {}

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.float().to(device), targets.to(device)

            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(float().torch.float32)
                predicted = (outputs > 0.5).float()
                correct += (predicted == targets).all(dim=1).sum().item()
            else:
                targets = targets.squeeze().long()
                _, predicted = outputs.max(1)
                correct += predicted.eq(targets).sum().item()
                
                # Calculate class-wise accuracy
                for true, pred in zip(targets, predicted):
                    if true.ndim == 0:
                        true = int(true.item())
                        pred = int(pred.item())
                    else:
                        true = tuple(true.cpu().numpy())
                        pred = tuple(pred.cpu().numpy())
                    
                    if true == pred:
                        class_correct[true] = class_correct.get(true, 0) + 1
                    class_total[true] = class_total.get(true, 0) + 1

            total += targets.size(0)
            all_targets.extend(targets.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())

    acc = 100. * correct / total
    class_acc = {k: 100. * v / class_total[k] for k, v in class_correct.items()}

    auc = compute_auc(all_targets, all_outputs, task)
    
    return acc, auc, class_acc


# Utils

In [10]:
@dataclass
class ModelResults:
    accuracy: float
    auc: float
    class_acc: Dict[int, float]

def create_data_transform(num_channels: int) -> transforms.Compose:
    stats = [0.5] * num_channels
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=stats, std=stats)
    ])

def setup_datasets(
    data: Dict,
    dataset_class: type,
    loader_type: str, 
    transform,
    batch_size: int = 20,
    shuffle: bool = False
) -> Tuple[DataLoader, DataLoader]:
    """
    Creates train and test datasets using the provided dataset class.
    
    Args:
        data: Dictionary containing train and test data
        dataset_class: The dataset class to instantiate
        loader_type: Type of data loader to use
        transform: Transformations to apply to the data
        batch_size: Batch size for the DataLoader
        shuffle: Whether to shuffle the data
        
    Returns:
        Tuple of (train_dataloader, test_dataloader)
    """
    train_dataset = dataset_class(
        data['train_images'],
        data['train_labels'],
        loader_type,
        transform=transform
    )
    
    test_dataset = dataset_class(
        data['test_images'],
        data['test_labels'],
        loader_type,
        transform=transform
    )
    
    return (
        DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle),
        DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)
    )

# Main

In [11]:
def train_and_evaluate(loaders_dict: Dict[str, Dict], dataset_style) -> Dict[str, ModelResults]:
    results = {}
    
    for loader_type in loaders_dict.values():
        print(f'Loader Type: {loader_type}')
        
        # Load and prepare data
        data = np.load(CONFIG['data_path'])
        transform = create_data_transform(CONFIG['num_channels'])
        
        # Setup model and datasets
        model = Net_28(CONFIG['num_channels'], CONFIG['num_classes']).to(CONFIG['device'])
        train_loader, test_loader = setup_datasets(data, DATASET[dataset_style], loader_type, transform)
        
        # Train and evaluate
        model_trained = train(model, train_loader, CONFIG['task'])
        acc, auc, class_acc = test(model_trained, test_loader, CONFIG['task'])
        
        results[loader_type] = ModelResults(
            accuracy=acc,
            auc=auc,
            class_acc=class_acc
        )
        
        del model, model_trained
            
    return results

def process_results(results: Dict, dataset_name: str, fold: int, dataset_style: str):
    class_results = []
    overall_results = []
    
    # Process class-wise results
    for label in range(CONFIG['num_classes']):
        result_dict = {'Dataset': dataset_name, 'Class': label}
        for loader_type, metrics in results.items():
            result_dict[f'Accuracy_{loader_type}'] = metrics.class_acc.get(label, 0)
        class_results.append(result_dict)
    
    # Process overall results
    overall_dict = {'Dataset': dataset_name}
    for loader_type, metrics in results.items():
        overall_dict[f'Overall_Accuracy_{loader_type}'] = metrics.accuracy
        overall_dict[f'Overall_AUC_{loader_type}'] = metrics.auc
    overall_results.append(overall_dict)

    # # Save results
    # pd.DataFrame(class_results).to_csv(
    #     f"{CONFIG['result_path']}{dataset_name}_class_results_fold_{fold}_{dataset_style}.csv",
    #     index=False
    # )
    # pd.DataFrame(overall_results).to_csv(
    #     f"{CONFIG['result_path']}{dataset_name}_overall_results_fold_{fold}_{dataset_style}.csv",
    #     index=False
    # )

def main(fold: int):
    dataset_name = CONFIG['data_path'].rsplit('/', 1)[-1].rsplit('.', 1)[0]
    print(f'DATA: {dataset_name}')
    print(f'FOLD: {fold}')

    for dataset_style in DATASET.keys():
        print(f'DATASET_STYLE: {dataset_style}')
        if dataset_style == 'Gaussian':
            results = train_and_evaluate(GAUSSIAN_STD_VALUES, dataset_style)
            process_results(results, dataset_name, fold, dataset_style)
        elif dataset_style == 'Perturbed':
            results = train_and_evaluate(PERTURBATION_VALUES, dataset_style)
            process_results(results, dataset_name, fold, dataset_style)
        elif dataset_style == 'Removal':
            results = train_and_evaluate(REMOVAL_VALUES, dataset_style)
            process_results(results, dataset_name, fold, dataset_style)
    print()

if __name__ == '__main__':
    for fold in range(CONFIG['num_folds']):
        main(fold)
        print()
        print()

DATA: retinamnist
FOLD: 0
DATASET_STYLE: Perturbed
Loader Type: None


100%|██████████| 1/1 [00:00<00:00,  1.86it/s]


Loader Type: one


100%|██████████| 1/1 [00:00<00:00,  2.03it/s]


Loader Type: 1


...

100%|██████████| 1/1 [00:00<00:00,  1.67it/s]


Loader Type: 90


100%|██████████| 1/1 [00:00<00:00,  1.73it/s]


Loader Type: 99


100%|██████████| 1/1 [00:00<00:00,  1.71it/s]